## Deep Learning Coding Project 2: Image Classification

Before we start, please put your **Chinese** name and student ID in following format:

Name, 0000000000 // e.g.) 傅炜, 2021123123

赵瀚宏, 2023040163

## Introduction

We will use Python 3, [NumPy](https://numpy.org/), and [PyTorch](https://pytorch.org/) for this coding project. The example code has been tested under the latest stable release version.

### Task

In this notebook, you need to train a model to classify images. Given an image, you need to distinguish its category,
e.g., whether it is a horse or an automobile. There are total 10 classes:
airplane, automobile, bird, cat, deer, dog, frog, horse, ship and truck. We
release 40,000 images for training, 10,000 images for validation. Each image has
a shape of (3, 128, 128). We will evaluate your model in 10,000 images on the test set.

Download the dataset from [here](https://cloud.tsinghua.edu.cn/d/00e0704738e04d32978b/) and organize them into a folder named "cifar_10_4x".

<!-- Images can be classified as "No Finding" or **one or more types**. In the basic task, given an image, you only need to tell whether the X-ray indicates "Infiltration". In the bonus task, you need to tell whether *each* of the diseases exists.

Images are taken from the [ChestX-ray14 dataset](https://www.kaggle.com/nih-chest-xrays/data) and downsampled to (256, 256). We release 44872 gray scale images for training and validation. We will evaluate your model on 10285 images in the test set. The dataset is available [here](https://cloud.tsinghua.edu.cn/d/16d06a89c5b4459db703/) and organized as follows: `train` directory includes all images for training and validation, and each line of `train.txt` records the labels separated by "|". -->

### Coding

We provide a code template. You can add new cells and modify our example to train your own model. To run this code, you should:

+ implement your model (named `Net`) in `model.py`.
+ implement your training loop in this notebook

Your final submitted model should not be larger than **20M**. **Using any pretrained model is NOT permitted**.
Besides, before you submit your result, **make sure you can test your model using our evaluation cell.** Name your best model "cifar10_4x_best.pth".

### Report & Submission

Your report should include:

1. the details of your model
2. all the hyper-parameters
3. all the tricks or training techniques you use
4. the training curve of your submitted model.

Reporting additional ablation studies and how you improve your model are also encouraged.

You should submit:

+ all codes
+ the model checkpoint (only "cifar10_4x_best.pth")
+ your report (a separate "pdf")

to web learning. We will use the evaluation code in this notebook to evaluate your model on the test set.

### Grading

We will grade this coding project based on the performance of your model (70%) and your report (30%). Regarding the evaluation metric of your model, assume your test accuracy is $X$, then your score is

$\frac{min(X,H)−0.6}{H−0.6}×7$

where $H$ is accuracy of the model trained by TAs and $H=0.9$, i.e., you will get the full score if your test accuracy is above 90%.

**Bonus**: The best submission with the highest testing accuracy will get 1 bonus point for the final course grade.

**Avoid plagiarism! Any student who violates academic integrity will be seriously dealt with and receive an F for the course.**

## Code Template

We have masked the the training loop in this notebook for you to complete. You should also overwrite "model.py" and implement your own model.

In [1]:
%load_ext autoreload
%autoreload 2

### Setup Code

If you use Colab in this coding project, please uncomment the code, fill the `GOOGLE_DRIVE_PATH_AFTER_MYDRIVE` and run the following cells to mount your Google drive. Then, the notebook can find the required file. If you run the notebook locally, you can skip the following cells.

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# import os

# # TODO: Fill in the Google Drive path where you uploaded the assignment
# # Example: If you create a 2022SP folder and put all the files under CP1 folder, then '2022SP/CP1'
# # GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = '2022SP/CP1'
# GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = None 
# GOOGLE_DRIVE_PATH = os.path.join('drive', 'MyDrive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
# print(os.listdir(GOOGLE_DRIVE_PATH))

In [4]:
# import sys
# sys.path.append(GOOGLE_DRIVE_PATH)

In [6]:
from dataset import CIFAR10_4x
from evaluation import evaluation

from model import Net  # this should be implemented by yourself

### Enjoy Your Coding Time!

In [7]:
import math
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import transforms
from PIL import Image


def set_seed(seed):
    seed = int(seed)
    if seed < 0 or seed > (2**32 - 1):
        raise ValueError("Seed must be between 0 and 2**32 - 1")
    else:
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True


device = 'cuda' if torch.cuda.is_available() else 'cpu'
set_seed(16)

In [8]:
data_root_dir = '.'

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([125 / 255, 124 / 255, 115 / 255],
                         [60 / 255, 59 / 255, 64 / 255])
])

trainset = CIFAR10_4x(root=data_root_dir,
                      split="train", transform=transform)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=32, shuffle=True, num_workers=8, pin_memory=True)

validset = CIFAR10_4x(root=data_root_dir,
                      split='valid', transform=transform)
validloader = torch.utils.data.DataLoader(
    validset, batch_size=128, shuffle=False, num_workers=8)


In [20]:
####################################
#                                  #
#                                  #
#                                  #
#                                  #
#        RUN THIS FOR NEW MODEL    #
#                                  #
#                                  #
#                                  #
#                                  #
####################################
net = Net()
print("number of trained parameters: %d" % (
    sum([param.nelement() for param in net.parameters() if param.requires_grad])))
print("number of total parameters: %d" %
      (sum([param.nelement() for param in net.parameters()])))

criterion = nn.CrossEntropyLoss()
train_history = []
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

net.to(device)

number of trained parameters: 119326
number of total parameters: 119326


Net(
  (resBlock1): Sequential(
    (0): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  )
  (resBlock2): Sequential(
    (0): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  )
  (resBlock3): Sequential(
    (0): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  )
  (first): Sequential(
    (0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine

In [10]:
# NO EXECUTION
model_dir = '.'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
torch.save(net, os.path.join(model_dir, 'cifar10_4x_0.pth'))

# check the model size
os.system(' '.join(['du', '-h', os.path.join(model_dir, 'cifar10_4x_0.pth')]))

484K	./cifar10_4x_0.pth


0

In [11]:
def train(epochs:int):
    ##############################################################################
    #                  TODO: You need to complete the code here                  #
    ##############################################################################
    # YOUR CODE HERE
    # print(optimizer.param_groups[0])
    results = []
    from tqdm import tqdm
    from evaluation import evaluation
    for epoch in range(epochs):
        with tqdm(trainloader) as bar:
            nums = trues = 0
            for i,(xb,yb) in enumerate(bar):
                xb,yb = xb.to(device),yb.to(device)
                optimizer.zero_grad()
                preds = net(xb)
                with torch.no_grad():
                    _, answer = torch.max(preds, 1)
                    trues += (answer==yb).sum().item()
                    nums += len(yb)
                loss = criterion(preds,yb)
                loss.backward()
                optimizer.step()
                if(i%100==0):
                    bar.set_description(f'Epoch {epoch}/{epochs}, accuracy {100*trues/nums}')
        # with tqdm(validloader) as bar:
        #     with torch.no_grad():
        #         for i,(xb,yb) in enumerate(bar):
        #             xb,yb = xb.to(device),yb.to(device)
        #             preds = net(xb)
        #             loss += criterion(preds,yb)
        # print(f'Epoch {epoch}/{epochs}, valid loss {loss}')
        train_acc = 100*trues/nums
        print(f'train accuracy: {train_acc}%')
        valid_acc =evaluation(net,validloader,device)
        print(f'valid accuracy: {valid_acc}%')
        results.append({'train accuracy':train_acc,'valid accuracy':valid_acc})
        train_history.append({
        'optimizer':{k:v for k,v in optimizer.param_groups[0].items() if k!='params'},
        'epochs':'no.'+str(len(results)),
        'results':results
    })
    # raise NotImplementedError()
    ##############################################################################
    #                              END OF YOUR CODE                              #
    ##############################################################################

In [12]:
torch.backends.cudnn.enabled = False

In [23]:
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3,weight_decay=1e-5)
train(5)

Epoch 0/5, accuracy 89.0820149875104: 100%|██████████| 1250/1250 [01:20<00:00, 15.61it/s] 

train accuracy: 88.9675%


Accuracy of the network on the valid images: 69 %
valid accuracy: 69.79%


Epoch 1/5, accuracy 90.00052039966694: 100%|██████████| 1250/1250 [01:20<00:00, 15.49it/s]

train accuracy: 89.985%


...

train accuracy: 91.21%


Accuracy of the network on the valid images: 70 %
valid accuracy: 70.48%


Epoch 4/5, accuracy 91.46024146544546: 100%|██████████| 1250/1250 [01:20<00:00, 15.57it/s]

train accuracy: 91.315%


Accuracy of the network on the valid images: 69 %
valid accuracy: 69.46%


In [24]:
from evaluation import evaluation
evaluation(net,validloader,device)

Accuracy of the network on the valid images: 69 %


69.46

In [25]:
# print(current_time)
model_path = os.path.join(model_dir,'models', f'cifar10_4x_best.pth')
torch.save(net, model_path)
assert(False)
try:
    torch.save(net, model_path)
except:
    torch.save(net.state_dict(),model_path)
    print('Saving has some errors')
# torch.save(net, os.path.join(model_dir, f'cifar10_4x_{time.localtime()}.pth'))

AssertionError: 

## Evaluation

Before submission, please run the following cell to make sure your model can be correctly graded.

In [26]:
!python evaluation.py

number of trained parameters: 119326
number of total parameters: 119326
can't load test set because [Errno 2] No such file or directory: '/home/zhh/MyFile/DL/CodingProject2/cifar_10_4x/test', load valid set now
Accuracy of the network on the valid images: 69 %


In [27]:
acc = 69
import time
current_time = time.strftime("%m%d%H%M")
os.rename(model_path,os.path.join(model_dir,'models', f'cifar10_4x_{current_time}_acc{acc}.pth'))
model_src = ''

with open('model.py','r') as f:
    model_src=f.read()

with open(os.path.join(model_dir,'models', f'cifar10_4x_{current_time}_acc{acc}.json'),'w') as f:
    import json
    json.dump({
        'model_type':model_src,
        'train_history':train_history,
        'description':"""
    ResNet tune, add BatchNorm and weightdeacy, train multiple epochs so that the trian accuracy is over 90%; but the test accuracy remains the same.
    """
    },f,indent=4)

: 